### Create Map Plot PyGSI

In [1]:
def _create_map_plot(lats, lons, data, metadata,
                     outdir, domain, projection,
                     title, date_title, xlabel,
                     ylabel, annotate_stats,
                     colorbar, cbar_label, grid):

    # Set evaluation type to either diff or magnitude
    eval_type = 'diff' if metadata['Diag Type'] in ['omf', 'oma'] \
        else 'magnitude'

    # Get variable specs
    spec_variable = _varspecs_name(metadata['Variable'])

    varspecs = VariableSpecs(variable=spec_variable,
                             eval_type=eval_type)
    metadata['Variable'] = varspecs.name
    
    # If no data, plot empty map
    if len(data) == 0:
        _no_data_map(mymap, Domain(domain), metadata, outdir)
        return
    
    # Generate plot object
    plotobj = MapScatter(latitude=lats,
                         longitude=lons,
                         data=data)

    plotobj.cmap = varspecs.cmap
    plotobj.vmin = varspecs.vmin
    plotobj.vmax = varspecs.vmax
    
    mymap = CreatePlot()
    mymap.plot_layers = [plotobj]
    mymap.projection = projection
    mymap.domain = domain
    mymap.add_map_features(['coastline'])
    
    # Add features to the plot
    # Colorbar
    if colorbar:
        label = f"{metadata['Variable'].capitalize()} ({varspecs.units})"
        cbar_label = label if cbar_label is None else cbar_label

        mymap.add_colorbar(label=cbar_label,
                           label_fontsize=12,
                           extend='both')
    # Titles
    labels = _get_labels(metadata)

    title = labels['title'] if title is None else title
    mymap.add_title(title, loc='left', fontsize=12)

    date_title = labels['date title'] if date_title is None else date_title
    mymap.add_title(label=date_title,
                    loc='right', fontsize=12,
                    fontweight='semibold')

    # X and Y labels
    xlabel = "Longitude" if xlabel is None else xlabel
    ylabel = "Latitude" if ylabel is None else ylabel

    mymap.add_xlabel(xlabel, fontsize=12)
    mymap.add_ylabel(ylabel, fontsize=12)

    # Annotate Stats
    if annotate_stats:
        stats_dict = _calculate_stats(data)
        # Make stats dict values str because when annotating,
        # they refuse to round??
        for key in stats_dict:
            val = str(np.round(stats_dict[key], 3))
            stats_dict[key] = val

        # Each domain changes so need to set
        # values for each domain
        yloc_dict = {
            'global': -0.12,
            'north america': -0.10,
            'conus': -0.11,
            'europe': -0.095
        }

        mymap.add_stats_dict(stats_dict, yloc=yloc_dict[domain],
                             fontsize=12)

    # Grid
    if grid:
        mymap.add_grid()
    
    
    fig = CreateFigure()
    fig.plot_list = [mymap]
    fig.create_figure()
    fig.save_figure(outdir + f"{labels['save file']}_map.png",
                    bbox_inches='tight', pad_inches=0.1)
    
    return
    

### No Data map

In [2]:
def _no_data_map(domain, projection, metadata, outdir):
    """
    Creates a plot with 'No Data' across the map if
    there is no data.
    """

    mymap = CreatePlot()
    mymap.projection = projection
    mymap.domain = domain
    mymap.add_map_features(['coastline'])
    
    # Titles
    labels = _get_labels(metadata)
    mymap.add_title(labels['title'], loc='left', fontsize=12)

    mymap.add_title(label=labels['date title'],
                    loc='right', fontsize=12,
                    fontweight='semibold')
    
    # Get center of map location
    lon1 = domain.extent[0]+180
    lon2 = domain.extent[1]+180
    xloc = (lon2 - ((lon2-lon1)/2)) - 180

    lat1 = domain.extent[2]+90
    lat2 = domain.extent[3]+90
    yloc = (lat2 - (lat2-lat1)/2) - 90

    # Plot text
    mymap.add_text(xloc, yloc, 'No Data', fontsize=32,
                   alpha=0.6, horizontalalignment='center')

    fig = CreateFigure()
    fig.plot_list = [mymap]
    fig.create_figure()
    fig.save_figure(outdir + f"{labels['save file']}_map.png",
                    bbox_inches='tight', pad_inches=0.1)
    fig.close_figure()

### Histogram

In [ ]:
def _create_hist_plot(data, metadata, outdir, color, legend,
                      grid, plot_mean, plot_zero, title,
                      date_title, xlabel, ylabel,
                      annotate_stats):
    """
    Function to create histogram plot.
    """
    
    # Grab variable specs
    spec_variable = _varspecs_name(metadata['Variable'])

    # Set evaluation type to either diff or magnitude
    eval_type = 'diff' if metadata['Diag Type'] in \
        ['omf', 'oma'] else 'magnitude'
    varspecs = VariableSpecs(variable=spec_variable,
                             eval_type=eval_type)
    metadata['Variable'] = varspecs.name

    # Plot special histogram for if len of data is 0 or 1
    if len(data) <= 1:
        _no_data_histogram(data, myplot, metadata, outdir)
        return
    
    # Get Stats
    stats_dict = _calculate_stats(data)

    # Get bins
    bins = _get_bins(eval_type, stats_dict)
    
    # Create histogram plot object
    histobj = Histogram(data)
    histobj.bins = bins
    histobj.color = color
    
    plot_layers = [histobj]
    
    if eval_type == 'diff' and plot_zero:
        zeroline = VerticalLine(0)
        zeroline.color = 'k'
        zeroline.linestyle = 'dashed'
        zeroline.linewidth = 1.5
        
        plot_layers.append(zeroline)
        
    if plot_mean:
        meanline = VerticalLine(0)
        meanline.color = 'red'
        meanline.linestyle = 'solid'
        meanline.linewidth = 1
        meanline.label = 'Mean'
        
        plot_layers.append(meanline)
        
    myplot = CreatePlot()
    myplot.plot_layers = plot_layers

    if legend:
        myplot.add_legend()

    if grid:
        myplot.add_grid()
    
    # Titles
    labels = _get_labels(metadata)

    title = labels['title'] if title is None else title
    myplot.add_title(title, loc='left', fontsize=12)

    date_title = labels['date title'] if date_title is None else date_title
    myplot.add_title(label=date_title,
                     loc='right', fontsize=12,
                     fontweight='semibold')
    
    # X and Y labels
    xlabel = f"{varspecs.name.capitalize()} ({varspecs.units})" \
        if xlabel is None else xlabel
    ylabel = "Count" if ylabel is None else ylabel

    myplot.add_xlabel(xlabel, fontsize=12)
    myplot.add_ylabel(ylabel, fontsize=12)
    
    if annotate_stats:
        # Make stats dict values str because when annotating,
        # they refuse to round??
        for key in stats_dict:
            val = str(np.round(stats_dict[key], 3))
            stats_dict[key] = val

        myplot.add_stats_dict(stats_dict, yloc=-0.15, fontsize=12)
        
    fig = CreateFigure()
    fig.plot_list = [mymap]
    fig.create_figure()
    fig.save_figure(outdir + f"{labels['save file']}_hist.png",
                    bbox_inches='tight', pad_inches=0.1)
    fig.close_figure()